# Problème de régression

##### Importation de la base

In [1]:
url2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/"
file2 = "winequality-white.csv"
import pyensae
data = pyensae.download_data(file2, website=url2)
import pandas
dfwhite = pandas.read_csv("winequality-white.csv",sep=";")
dfwhite.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6


##### Importation des packages nécessaires

In [2]:
import pandas
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import prettyplotlib as ppl
import random


##### Observation de la distribution des qualités

In [3]:
#Nombre d'occurrences dans la base complète de chaque qualité
dfwhite.quality.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
dtype: int64

In [4]:
#Histogramme
from pylab import *
plt.style.use('ggplot')
pandas.DataFrame.hist(dfwhite, column="quality", bins=7, color='indianred', grid=False)
show()

In [5]:
#Couleurs disponibles
import matplotlib.patches as patches
import matplotlib.colors as colors
import math


fig = plt.figure()
ax = fig.add_subplot(111)

ratio = 1.0 / 3.0
count = math.ceil(math.sqrt(len(colors.cnames)))
x_count = count * ratio
y_count = count / ratio
x = 0
y = 0
w = 1 / x_count
h = 1 / y_count

for c in colors.cnames:
    pos = (x / x_count, y / y_count)
    ax.add_patch(patches.Rectangle(pos, w, h, color=c))
    ax.annotate(c, xy=pos)
    if y >= y_count-1:
        x += 1
        y = 0
    else:
        y += 1

plt.show()

##### Séparation de la base en une base d'entraînement et une base de test

In [6]:
from sklearn.cross_validation import train_test_split

#Pour que la même base de test sorte à chaque fois qu'on exécute le programme, on utilise : random_state=un nombre entier quelconque choisi arbitrairement

X_train, X_test, y_train, y_test = train_test_split(dfwhite[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']],dfwhite.quality, random_state=3)

In [7]:
#Vérification de la taille de la base de test
len(y_test)/len(dfwhite)
#On doit trouver 25% qui est le paramètre par défaut dans scikit learn

0.250102082482646

## Régression linéaire simple (Chapitre 2)

### Régression linéaire simple

In [8]:
#Entraînement du modèle sur la base d'apprentissage
from sklearn.linear_model import LinearRegression

reglin = LinearRegression(normalize=True)
#On utilise normalize=True car les variables ont des ordres de grandeurs très différents

reglin.fit(X_train, y_train)

reglin.coef_, reglin.intercept_, "R^2=",reglin.score(X_train, y_train)

(array([  7.08672075e-02,  -1.91050896e+00,   3.92700519e-02,
          8.24994938e-02,  -3.62076438e-02,   3.79589490e-03,
         -5.95091904e-04,  -1.47240724e+02,   7.30192091e-01,
          6.56917370e-01,   1.94989276e-01]),
 146.99034335580782,
 'R^2=',
 0.28213632011280609)

In [9]:
#Prédiction de la qualité sur la base de test
predicted_reglin = reglin.predict(X_test)
expected_reglin = y_test

In [10]:
#Taux de bonnes réponses
nb_goodpred_reglin=0
for i in range(0,len(predicted_reglin)) :
    if round(predicted_reglin[i])==expected_reglin[i] :
        nb_goodpred_reglin=nb_goodpred_reglin+1
print(nb_goodpred_reglin/(len(predicted_reglin)+1))


0.5179445350734094


In [11]:
#Erreur quadratique totale
e_reglin_2=((predicted_reglin-expected_reglin)**2).sum()
print(e_reglin_2)

701.192723457


In [12]:
#Erreur absolue totale
e_reglin_1=abs(predicted_reglin-expected_reglin).sum()
print(e_reglin_1)
#Erreur absolue moyenne
print(e_reglin_1/len(y_test))

719.379822215
0.587248834462


In [13]:
#Erreur sans prendre en compte les erreurs absolues inférieures à epsilon = 0.5
e_reglin_epsilon1=sum(abs(predicted_reglin[i]-expected_reglin[i]) for i in range(0, len(y_test)) if abs(predicted_reglin[i]-expected_reglin[i])>0.5)
print(e_reglin_epsilon1)

557.744327654


In [14]:
#Erreur sans prendre en compte les erreurs absolues inférieures ou égales à epsilon = 1
e_reglin_epsilon2=sum(abs(predicted_reglin[i]-expected_reglin[i]) for i in range(0, len(y_test)) if abs(predicted_reglin[i]-expected_reglin[i])>1)
print(e_reglin_epsilon2)

282.542109538


In [16]:
#Introduction de bruit et arrondi à .1 des prédictions pour mieux voir les résultats sur les graphes suivants

expected_reglin_noise=expected_reglin+np.random.uniform(0,0.2,size=len(y_test))


predicted_reglin_simplified=np.around(predicted_reglin,1)

In [17]:
#Graphe hexbin avec du bruit et arrondi à 0.1 près des prédictions : comparaison des qualités prédites et des qualités réelles

plt.hexbin(expected_reglin_noise,predicted_reglin_simplified, cmap=plt.cm.YlGn)
plt.xlabel('Vraie qualité')
plt.ylabel('Qualité prédite')
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.plot([4.5,7.2],[4.5,7.2],'--k')
plt.show()

In [18]:
#Graphe scatter avec bruit et arrondi à 0.1 près des prédictions pour mieux voir les prédictions des qualités en faible proportion dans la base de test
ppl.scatter(expected_reglin_noise, predicted_reglin_simplified)
ppl.plot([3,9],[3,9],'--k')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Qualité prédite')
show()

In [19]:
#Graphique : Erreur absolue selon caractéristique chimique (à décliner selon la variable voulue)
ppl.scatter(x=X_test[:,7],y=abs(expected_reglin-predicted_reglin), color='green')

plt.axis('tight')
plt.title('Density')
plt.xlabel('Density')
plt.ylabel('Erreur en valeur absolue')
plt.show()

In [20]:
#Graphique : Erreur absolue selon caractéristique chimique (à décliner selon la variable voulue)
#Avec les couleurs, on voit ainsi mieux si les pics d'erreurs sur les graphiques précédents ne sont pas tout simplement dûs au fait qu'il y a plus de vins présentant des valeurs similaires pour la variable considérée et donc il y a plus de "chances" que les vins tres mal prédits présentent aussi cette valeur pour la variable considérée 
ppl.scatter(x=arange(0,len(y_test)),y=abs(expected_reglin-predicted_reglin), c=X_test[:,10])

plt.axis('tight')
plt.title('Alcohol')
plt.xlabel('Vins blans')
plt.ylabel('Erreur en valeur absolue')
plt.show()

In [21]:
#Graphe: erreur absolue selon la qualité espérée
e_reglin_plot=abs(predicted_reglin-expected_reglin)

ppl.scatter(expected_reglin_noise, e_reglin_plot, color='indianred')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur absolue')
plt.show()

###  ANNEXE : En enlevant la variable pH:

In [22]:
#On utilise la même base d'apprentissage et la même base de test avec pour seule différence qu'elles ne contiennent plus la colonne pH
X_train_bis=np.concatenate((X_train[:,0:8],X_train[:,9:11]), axis=1)
X_test_bis=np.concatenate((X_test[:,0:8],X_test[:,9:11]), axis=1)
print(X_train.shape,X_train_bis.shape)

(3673, 11) (3673, 10)


In [23]:
#Entraînement du modèle sur la "nouvelle" base d'apprentissage qui ne contient plus pH
from sklearn.linear_model import LinearRegression

reglin = LinearRegression(normalize=True)

reglin.fit(X_train_bis, y_train)

reglin.coef_, reglin.intercept_, "R^2=",reglin.score(X_train_bis, y_train)

(array([ -2.44835323e-02,  -1.99471791e+00,  -1.29122247e-02,
          5.11683132e-02,  -6.19574206e-01,   4.07787971e-03,
         -6.04879330e-04,  -7.01651721e+01,   6.26933940e-01,
          2.83162184e-01]), 72.702959038751047, 'R^2=', 0.27511073453404677)

In [24]:
#Prédiction sur la base de test
predicted_reglin_bis = reglin.predict(X_test_bis)
expected_reglin_bis = y_test

In [25]:
#Taux de bonnes réponses
nb_goodpred_reglin_bis=0
for i in range(0,len(predicted_reglin_bis)) :
    if round(predicted_reglin_bis[i])==expected_reglin_bis[i] :
        nb_goodpred_reglin_bis=nb_goodpred_reglin_bis+1
print(nb_goodpred_reglin_bis/(len(predicted_reglin_bis)+1))


0.5154975530179445


In [26]:
#Erreur quadratique sur la base de test
e_reglin=((predicted_reglin_bis-expected_reglin_bis)**2).sum()
print(e_reglin)

704.663149911


In [27]:
#Graphe hexbin : comparaison des prédictions aux qualités réelles

plt.hexbin(expected_reglin_bis,predicted_reglin_bis, cmap=plt.cm.YlGn)
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.show()

-On n'observe pas de différence fondamentale de performance. (même taux de bonnes réponses et erreur quadratique similaire) 

-Néanmoins les coefficients obtenus sont très différents de précédemment, certains ont même un signe différent.

-Il faudrait alors estimer la variance des coefficients de la droite de régression et regarder si elle est véritablement plus faible pour pouvoir choisir entre les deux modèles celui qui est le plus "stable" lorsqu'on change de base d'apprentissage.
L'autre possibilité, qui est mise en place plus bas, est d'utiliser une régression Ridge pour de toute façon réduire la variance des coefficients quitte à introduire un biais. Ceci permet de ne pas s'occuper de la multicolinéarité liée à la variable pH mais aussi celle des autres variables (total sulfur dioxide et free sulfur dioxide par exemple)

-Rien qu'en testant sur plusieurs bases de tests (en enlevant random state à l'étape train test split et en rééxécutant le programme plusieurs fois), on remarque que la performance ne change pas vraiment. Je n'ai donc pas creusé dans cette direction, j'ai plutôt essayé d'améliorer la performance avant la "stabilité" des prédictions sur des bases d'apprentissages différentes.

### ANNEXE : En enlevant des vins de qualité moyenne: (Attention on n'opère ici plus du tout sur les mêmes bases train et text)

J'ai voulu regarder ici, puisque les vins de qualité extrêmes étaient très mal prédits, si en rééquilibrant les proportions des différents vins, la droite de régression ne s'ajustait pas différemment en "prenant plus en compte" les vins de qualité extrême.

In [29]:
#On crée une nouvelle variable taken qui prend au hasard l'un des entiers parmi 0, 1 et 2 lorsque le vin est de qualité moyenne. Lorsque le vin est de qualité extrême, taken prend nécessairement la valeur 0.
#On ne garde ensuite que les vins qui ont pris la modalité 0 pour la variable taken. On supprime ainsi environ deux tiers des vins de qualité moyenne de la base.
import random
dfwhite_lisse=dfwhite.copy()


dfwhite_lisse['taken']=dfwhite_lisse['quality'].apply(lambda x: 0 if (x<5|x>7) else random.randint(0,3))
dfwhite_lisse=dfwhite_lisse[dfwhite_lisse['taken']==0]
dfwhite_lisse.shape


(1369, 13)

In [30]:
#On réutilise train test split, ici on n'opère donc plus sur une base de test similaire : les vins de la nouvelle base de test ne seront pas nécessairement dans la base de test précédente.
#Ce n'est pas très grave car l'idée est déjà de voir ici s'il y a une réelle différente de performance dans la prédiction des vins de qualité extrême.
from sklearn.cross_validation import train_test_split

X_train_lisse, X_test_lisse, y_train_lisse, y_test_lisse = train_test_split(dfwhite_lisse[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']],dfwhite_lisse.quality, random_state=3)

In [31]:
#Entraînement du modèle
from sklearn.linear_model import LinearRegression

reglin_lisse = LinearRegression(normalize=True)

reglin_lisse.fit(X_train_lisse, y_train_lisse)

reglin_lisse.coef_, reglin_lisse.intercept_, "R^2=",reglin_lisse.score(X_train_lisse, y_train_lisse)

(array([  1.45909085e-01,  -1.81684845e+00,   2.79947760e-01,
          1.35410747e-01,   9.61365174e-01,   4.60806744e-03,
          1.58178490e-04,  -2.52620494e+02,   1.58050712e+00,
          7.75520304e-01,   1.92912671e-01]),
 248.00141567425661,
 'R^2=',
 0.33187780050260518)

In [32]:
predicted_reglin_lisse=reglin_lisse.predict(X_test_lisse)
expected_reglin_lisse=y_test_lisse

In [33]:
#Taux de bonnes réponses
nb_goodpred_reglin_lisse=0
for i in range(0,len(predicted_reglin_lisse)) :
    if round(predicted_reglin_lisse[i])==expected_reglin_lisse[i] :
        nb_goodpred_reglin_lisse=nb_goodpred_reglin_lisse+1
print(nb_goodpred_reglin_lisse/(len(predicted_reglin_lisse)+1))

0.45348837209302323


In [34]:
#Graphe hexbin : Comparaison des prédictions aux qualités réelles

plt.hexbin(expected_reglin_lisse,predicted_reglin_lisse, cmap=plt.cm.YlGn)
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.show()

In [35]:
#Graphe : erreur absolue selon la qualité espérée
e_reglin_plot=abs(predicted_reglin-expected_reglin)

ppl.scatter(expected_reglin, e_reglin_plot, color='indianred')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur absolue')
plt.show()

#On remarque que les vins de qualité extrême sont toujours mal prédits...

Une distribution plus équitable des vins n'améliorent pas la performance, au contraire cela la dégrade légèrement car les vins de qualité moyenne sont moins bien prédits. J'interprète ce résultat comme le fait que les vins de qualité extrême présents dans la base semblent ne pas avoir suffisamment de caractéristiques communes, donc ou bien il n'y en a pas assez dans la base de données et donc le modèle ne réussit pas à les déterminer ou bien la notation à l'extrême est trop subjective pour être prédite ou bien il manque des informations (des variables) qui expliqueraient une note extrême.

## Régression Ridge (Chapitre 3)

### ANNEXE : Régression Ridge (linéaire) + Cross-validation pour trouver l'hyperparamètre alpha

In [36]:
#Entraînement du modèle Ridge linéaire avec validation croisée
from sklearn import linear_model
ridgereg = linear_model.RidgeCV(alphas=[0.1, 0.01,0.00001], normalize=True)

ridgereg.fit(X_train, y_train)

ridgereg.coef_, ridgereg.intercept_ , "R^2=",ridgereg.score(X_train, y_train), ridgereg.alpha_ 

(array([  3.85997187e-02,  -1.91019639e+00,   3.55084364e-02,
          6.67786859e-02,  -3.21307060e-01,   4.05924028e-03,
         -7.22180739e-04,  -1.09090542e+02,   5.81707383e-01,
          5.95690521e-01,   2.32119094e-01]),
 109.5251476707829,
 'R^2=',
 0.28142497872855043,
 0.01)

In [37]:
#Prédiction à partir de la base de test
predicted_ridgereg = ridgereg.predict(X_test)
expected_ridgereg = y_test

In [38]:
#Taux de bonnes réponses
nb_goodpred_ridgereg=0
for i in range(0,len(predicted_ridgereg)) :
    if round(predicted_ridgereg[i])==expected_ridgereg[i] :
        nb_goodpred_ridgereg=nb_goodpred_ridgereg+1
print(nb_goodpred_ridgereg/(len(predicted_ridgereg)+1))


0.5203915171288744


In [39]:
#Erreur quadratique
e_ridgereg2=((predicted_ridgereg-expected_ridgereg)**2).sum()
print(e_ridgereg2)

701.451787903


In [40]:
#Erreur absolue
e_ridgereg1=abs(predicted_ridgereg-expected_ridgereg).sum()
print(e_ridgereg1)

718.77317785


In [41]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 0.5
e_ridgereg_epsilon1=sum(abs(predicted_ridgereg[i]-expected_ridgereg[i]) for i in range(0, len(y_test)) if abs(predicted_ridgereg[i]-expected_ridgereg[i])>0.5)
print(e_ridgereg_epsilon1)

556.128634637


In [42]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 1
e_ridgereg_epsilon1=sum(abs(predicted_ridgereg[i]-expected_ridgereg[i]) for i in range(0, len(y_test)) if abs(predicted_ridgereg[i]-expected_ridgereg[i])>1)
print(e_ridgereg_epsilon1)

282.968594452


In [43]:
#Graphe hexbin : Comparaison des prédictions aux qualités réelles
plt.hexbin(expected_ridgereg,predicted_ridgereg, cmap=plt.cm.YlGn)
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.show()

In [44]:
#Introduction de bruit sur les qualités "vraies" pour mieux voir les graphes et arrondi des préditions à un chiffre après la virgule pour des couleurs plus pertinentes
expected_ridgereg_noise=expected_ridgereg+np.random.uniform(0,0.2,size=len(y_test))


predicted_ridgereg_simplified=np.around(predicted_ridgereg,1)
print(predicted_ridgereg_simplified)

[ 5.5  5.6  6.  ...,  5.6  5.7  5.4]


In [45]:
#Graphe erreur absolue selon la qualité espérée
e_ridgereg_plot=abs(predicted_ridgereg-expected_ridgereg)

ppl.scatter(expected_ridgereg_noise, e_ridgereg_plot)
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur')
plt.show()

### ANNEXE : Comparaison régresssion linéaire et régression Ridge (linéaire) => AUCUNE DIFFERENCE MAJEURE

In [46]:
#Graphe : comparaison des 2 prédictions
plt.scatter(predicted_reglin, predicted_ridgereg)
plt.plot([3,9],[3,9],'--k')
plt.axis('tight')
plt.xlabel('Prédiction régression linéaire')
plt.ylabel('Prédiction régression Ridge linéaire')
plt.show()

In [47]:
#Graphe : erreur absolue selon la qualité espérée
e_ridgereg_plot=abs(predicted_ridgereg-expected_ridgereg)
e_reglin_plot=abs(predicted_reglin-expected_reglin)

ppl.scatter(expected_ridgereg_noise, e_ridgereg_plot)
ppl.scatter(expected_reglin_noise, e_reglin_plot)
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur')
plt.show()


Comme les différences ne sont pas très grandes en termes de performance et surtout de prédictions, même si normalement la régression Ridge devrait avoir des variances de coefficients moins élevés (ce que je n'ai pas vérifié ici), je n'ai pas développé ce modèle dans le rapport.

### Transformation de X par une fonction de base polynomiale (degré 2)

In [48]:
#On peut ici changer le degré facilement en changeant les nombres parcouru par j dans la boucle for. J'ai sélectionné 2 car les performances se dégradent quand on ajoute le degré 3.
X_train_poly=X_train.copy()
X_test_poly=X_test.copy()

for j in range(2,3):
    X_train_deg=X_train.copy()
    X_test_deg=X_test.copy()
    for i in range(0,11):
        X_train_deg[:, i]=X_train_deg[:, i]**j
        X_test_deg[:, i]=X_test_deg[:, i]**j
    X_train_poly=np.concatenate((X_train_poly,X_train_deg), axis=1)
    X_test_poly=np.concatenate((X_test_poly,X_test_deg), axis=1)
    
X_train_poly.shape, X_train.shape, X_test_poly.shape, X_test.shape

((3673, 22), (3673, 11), (1225, 22), (1225, 11))

### Régression Ridge polynomiale de degré 2

In [49]:
#Entraînement du modèle sur la base d'apprentissage avec validation croisée en même temps. 
#NB : Je n'ai laissé que ces alpha à tester après d'autres essais avec des alphas supérieurs qui n'étaient pas choisis pour que ça ne prenne pas trop de temps à réexécuter
from sklearn import linear_model
ridgereg_poly = linear_model.RidgeCV(alphas=[0.01, 0.1, 0.2, 0.3], normalize=True)

ridgereg_poly.fit(X_train_poly, y_train)


RidgeCV(alphas=[0.01, 0.1, 0.2, 0.3], cv=None, fit_intercept=True,
    gcv_mode=None, loss_func=None, normalize=True, score_func=None,
    scoring=None, store_cv_values=False)

In [50]:
ridgereg_poly.coef_, ridgereg_poly.intercept_ , "R^2=",ridgereg_poly.score(X_train_poly, y_train), ridgereg_poly.alpha_ 

(array([  2.18322137e-02,  -1.34587489e+00,   4.19822691e-01,
          3.13731056e-02,  -1.82919238e+00,   1.00351748e-02,
          9.55537480e-04,  -2.68094383e+01,   1.67062638e-01,
          2.64614983e-01,   1.19984375e-01,  -2.19378166e-03,
         -5.48978962e-01,  -5.05219848e-01,   3.65203150e-04,
          4.17705606e+00,  -6.18534057e-05,  -6.70087308e-06,
         -1.33375828e+01,   2.94131388e-02,   2.16788094e-01,
          6.62018958e-03]),
 42.580267788047962,
 'R^2=',
 0.2968238780703979,
 0.10000000000000001)

In [51]:
#Prédictions sur la base de test
predicted_ridgereg_poly = ridgereg_poly.predict(X_test_poly)
expected_ridgereg_poly = y_test

In [52]:
#Graphe scatter : comparaison prédictions aux qualités réelles
ppl.scatter(expected_ridgereg_poly, predicted_ridgereg_poly, color='indianred')
ppl.plot([3,9],[3,9],'--k')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Qualité prédite')
plt.show()

In [53]:
#Taux de bonnes réponses
nb_goodpred_ridgereg_poly=0
for i in range(0,len(predicted_ridgereg_poly)) :
    if round(predicted_ridgereg_poly[i])==expected_ridgereg_poly[i] :
        nb_goodpred_ridgereg_poly=nb_goodpred_ridgereg_poly+1
print(nb_goodpred_ridgereg_poly/(len(predicted_ridgereg_poly)+1))


0.5244698205546493


In [54]:
#Erreur quadratique totale(utilisée dans l'apprentissage)
e_ridgereg_poly2=((predicted_ridgereg_poly-expected_ridgereg_poly)**2).sum()
print(e_ridgereg_poly2)

690.52275081


In [55]:
#Erreur absolue totale
e_ridgereg_poly1=abs(predicted_ridgereg_poly-expected_ridgereg_poly).sum()
print(e_ridgereg_poly1)
#Erreur absolue moyenne
print(e_ridgereg_poly1/len(y_test))

713.164555609
0.582175147436


In [56]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 0.5
e_ridgereg_poly_epsilon1=sum(abs(predicted_ridgereg_poly[i]-expected_ridgereg_poly[i]) for i in range(0, len(y_test)) if abs(predicted_ridgereg_poly[i]-expected_ridgereg_poly[i])>0.5)
print(e_ridgereg_poly_epsilon1)

549.246972914


In [57]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 
e_ridgereg_poly_epsilon2=sum(abs(predicted_ridgereg_poly[i]-expected_ridgereg_poly[i]) for i in range(0, len(y_test)) if abs(predicted_ridgereg_poly[i]-expected_ridgereg_poly[i])>1)
print(e_ridgereg_poly_epsilon2)

272.663199003


In [58]:
#Introduction de bruit sur les qualités "vraies" pour mieux voir les graphes et arrondi des préditions à un chiffre après la virgule pour des couleurs plus pertinentes
expected_ridgereg_poly_noise=expected_ridgereg_poly+np.random.uniform(0,0.2,size=len(y_test))


predicted_ridgereg_poly_simplified=np.around(predicted_ridgereg_poly,1)

In [59]:
#Graphe hexbin : Comparaison des prédictions aux qualités réelles
plt.hexbin(expected_ridgereg_poly_noise,predicted_ridgereg_poly_simplified, cmap=plt.cm.YlGn)
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
plt.plot([4.5,7.1],[4.5,7.1],'--k')
cb.set_label('Nombre de prédictions')
plt.show()

In [60]:
#Prédiction selon le taux d'alcool : les vins à taux d'alcool plus élevé sont prédits avec une qualité supérieure
plt.scatter(expected_ridgereg_poly_noise, predicted_ridgereg_poly_simplified, c=X_test[:,10])
plt.plot([3,9],[3,9],'--k')
cb = plt.colorbar()
cb.set_label('Alcohol')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Qualité prédite')
plt.show()

In [61]:
#Graphe selon caractéristique chimique (à décliner selon la variable voulue)
ppl.scatter(x=X_test[:,3],y=abs(expected_ridgereg_poly-predicted_ridgereg_poly), color='lightslategray')
plt.axis('tight')
plt.title('Residual sugar')
plt.xlabel('Residual sugar')
plt.ylabel('Erreur absolue')
plt.show()

In [62]:
#Graphique : Erreur absolue selon caractéristique chimique (à décliner selon la variable voulue)
#Avec les couleurs, on voit ainsi mieux si les pics d'erreurs sur les graphiques précédents ne sont pas tout simplement dûs au fait qu'il y a plus de vins présentant des valeurs similaires pour la variable considérée et donc il y a plus de "chances" que les vins tres mal prédits présentent aussi cette valeur pour la variable considérée 
ppl.scatter(x=arange(0,len(y_test)),y=abs(expected_ridgereg_poly-predicted_ridgereg_poly), c=X_test[:,10])

plt.axis('tight')
plt.title('Alcohol')
plt.xlabel('Vins blans')
plt.ylabel('Erreur en valeur absolue')
plt.show()

##### Comparaison régression linéaire et régression ridge polynomiale de degré 2

In [63]:
#Comparaison des prédictions régression linéaire simple et régression Ridge poly de degré 2
plt.scatter(predicted_reglin, predicted_ridgereg_poly)
plt.plot([3,9],[3,9],'--k')
plt.axis('tight')
plt.xlabel('Prédiction régression linéaire')
plt.ylabel('Prédiction régression Ridge polynomiale')
plt.show()
#Il y a déjà plus de différences qu'entre la régression linéaire simple et la régression Ridge linéaire

In [64]:
#Graphe : erreur absolue selon la qualité espérée
e_ridgereg_poly_plot=abs(predicted_ridgereg_poly-expected_ridgereg_poly)
e_reglin_plot=abs(predicted_reglin-expected_reglin)

ppl.scatter(expected_ridgereg_poly_noise, e_ridgereg_poly_plot, color='green', label='Régression Ridge polynomiale')
ppl.scatter(expected_reglin_noise, e_reglin_plot,color='orange', label='Régression linéaire classique')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur')
plt.legend()
plt.show()


In [65]:
#Graphe taux d'erreur - évolution
e_sorted_reglin=sorted(e_reglin_plot)
e_sorted_ridgereg_poly=sorted(e_ridgereg_poly_plot)

ppl.plot(np.arange(0,len(y_test)), e_sorted_reglin, color='orange')
ppl.plot(np.arange(0,len(y_test)), e_sorted_ridgereg_poly, color='green')
plt.axis('tight')
plt.xlabel('')
plt.ylabel('Erreur')
plt.show()


### ANNEXE : Transformation de X par une fonction de base gaussienne

Ceci n'est pas détaillé dans le rapport car ça ne fait pas vraiment changer la performance par rapport à la régression linéaire simple. Elle semble même se dégrader comme l'indique le taux de bonnes réponses.

In [66]:
X_train_gauss=X_train.copy()
X_test_gauss=X_test.copy()

for i in range(0,11):
    X_train_gauss[:, i]=exp(-((X_train_gauss[:, i]-mean(X_train_gauss[:, i]))**2)/(2*var(X_train_gauss[:, i])))
    X_test_gauss[:, i]=exp(-((X_test_gauss[:, i]-mean(X_test_gauss[:, i]))**2)/(2*var(X_test_gauss[:, i])))

In [67]:
#Entraînement
from sklearn import linear_model
ridgereg_gauss = linear_model.RidgeCV(alphas=[0.01, 0.1, 0.5, 1,2,3,4,5,6,7,8,9], normalize=True)

ridgereg_gauss.fit(X_train_gauss, y_train)

ridgereg_gauss.coef_, ridgereg_gauss.intercept_ , "R^2=",ridgereg_gauss.score(X_train_gauss, y_train), ridgereg_gauss.alpha_ 

(array([ 0.07245023, -0.06919286,  0.70975267,  0.53486636,  0.11265204,
         0.49337973,  0.4118309 , -0.49470413, -0.20001275, -0.28571773,
        -0.42408889]), 5.194219212727444, 'R^2=', 0.17061109394577323, 0.01)

In [68]:
#Prédiction
predicted_ridgereg_gauss = ridgereg_gauss.predict(X_test_gauss)
expected_ridgereg_gauss = y_test

In [69]:
#Taux de bonnes réponses
nb_goodpred_ridgereg_gauss=0
for i in range(0,len(predicted_ridgereg_gauss)) :
    if round(predicted_ridgereg_gauss[i])==expected_ridgereg_gauss[i] :
        nb_goodpred_ridgereg_gauss=nb_goodpred_ridgereg_gauss+1
print(nb_goodpred_ridgereg_gauss/(len(predicted_ridgereg_gauss)+1))


0.4698205546492659


### ANNEXE : Bagging sur la régression polynomiale de degré 2

Le principe du bagging est :
- On génére  plusieurs échantillons bootstrap à partir de la base d'apprentissage (tirages avec remise à partir de la base d'apprentissage initiale où chaque vin a la même probabibilité d'être tiré) - avec la commande de scikit learn par défaut on créée 10 échantillons bootstrap

- A partir de ces nouveaux échantillons, on entraîne pour chaque nouvel échantillon un modèle Ridge avec alpha=0.1 (hyperparamètre trouvé précédemment)

- On prédit les qualités des vins de la base de test avec chacun de ces modèles (par défaut on a donc 10 prédictions par vin de la base de test)

- La prédiction finale est une moyenne de toutes ces prédictions

Objectifs : éviter le surapprentissage, éviter la variance des prédictions, on espère ainsi obtenir de meilleures prédictions

Néanmoins, ce n'est pas vraiment le cas ici, je n'ai donc pas détaillé cet essai dans le rapport.

In [70]:
#Entraînement
from sklearn.ensemble import BaggingRegressor
from sklearn import linear_model

bagging = BaggingRegressor(linear_model.Ridge(alpha=0.1, normalize=True),random_state=99)
#On utilise le alpha obtenu par validation croisée précédemment

bagging.fit(X_train_poly, y_train)

BaggingRegressor(base_estimator=Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, solver='auto', tol=0.001),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=99, verbose=0)

In [71]:
bagging.score(X_train_poly, y_train)

0.2961443595305554

In [72]:
#Prédiction
predicted_bagging = bagging.predict(X_test_poly)
expected_bagging = y_test

In [73]:
#Taux de bonnes réponses
nb_goodpred_bagging=0
for i in range(0,len(predicted_bagging)) :
    if round(predicted_bagging[i])==expected_bagging[i] :
        nb_goodpred_bagging=nb_goodpred_bagging+1
print(nb_goodpred_bagging/(len(predicted_bagging)+1))
#Comme prévu avec le R2 aucune amélioration des prédictions

0.5236541598694943


In [74]:
#Erreur quadratique
e_bagging=((predicted_bagging-expected_bagging)**2).sum()
print(e_bagging)

690.701724707


## Régression à vecteurs de support (Chapitre 4)

### ANNEXE : Régression à vecteurs de support (SVR) linéaire

Elle n'est pas détaillée dans le rapport car elle obtient une moins bonne performance que la SVR avec noyau gaussien.

In [75]:
#Entraînement
#NB :  je n'ai pas fait de validation croisée pour choisir C et epsilon mais on voit rapidement que la performance est quasi identique à précédemment
from sklearn import svm
svrlin=svm.SVR(C=2.0, epsilon=0.5, kernel='linear', random_state=16)
svrlin.fit(X_train,y_train)

SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.5, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=16,
  shrinking=True, tol=0.001, verbose=False)

In [76]:
svrlin.score(X_train, y_train)

0.26624845716430845

In [77]:
#Prédiction
predicted_svrlin = svrlin.predict(X_test)
expected_svrlin = y_test

In [78]:
#Taux de bonnes réponses: similaire au taux obtenu avec la régression Ridge polynomiale
nb_goodpred_svrlin=0
for i in range(0,len(predicted_svrlin)) :
    if round(predicted_svrlin[i])==expected_svrlin[i] :
        nb_goodpred_svrlin=nb_goodpred_svrlin+1
print(nb_goodpred_svrlin/(len(predicted_svrlin)+1))

0.5203915171288744


In [79]:
#Erreur quadratique 
e_svrlin=((predicted_svrlin-expected_svrlin)**2).sum()
print(e_svrlin)

709.517836111


In [80]:
#Erreur absolue
e_svrlin=(abs(predicted_svrlin-expected_svrlin)).sum()
print(e_svrlin)

724.193256203


In [81]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 0.5
e_svrlin_epsilon1=sum(abs(predicted_svrlin[i]-expected_svrlin[i]) for i in range(0, len(y_test)) if abs(predicted_svrlin[i]-expected_svrlin[i])>0.5)
print(e_svrlin_epsilon1)

556.407834013


In [82]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 1
e_svrlin_epsilon2=sum(abs(predicted_svrlin[i]-expected_svrlin[i]) for i in range(0, len(y_test)) if abs(predicted_svrlin[i]-expected_svrlin[i])>1)
print(e_svrlin_epsilon2)

281.633542928


In [83]:
#Graphe hexbin : Comparaison des prédictions aux qualités réelles
plt.hexbin(expected_svrlin,predicted_svrlin, cmap=plt.cm.YlGn)
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.show()

In [84]:
#Introduction de bruit sur les qualités "vraies" pour mieux voir les graphes et arrondi des préditions à un chiffre après la virgule
expected_svrlin_noise=expected_svrlin+np.random.uniform(0,0.2,size=len(y_test))

predicted_svrlin_simplified=np.around(predicted_svrlin,1)

In [85]:
#Graphe du taux d'erreur selon la qualité espérée
e_ridgereg_poly_plot=(predicted_ridgereg_poly-expected_ridgereg_poly)**2
e_svrlin_plot=(predicted_svrlin-expected_svrlin)**2

import prettyplotlib as ppl
import matplotlib.pyplot as plt
ppl.scatter(expected_svrlin_noise, e_svrlin_plot, color='green')
ppl.scatter(expected_ridgereg_poly_noise, e_ridgereg_poly_plot,color='orange')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur')
plt.show()

### Régression à vecteurs de support avec noyau gaussien (parties 4.1 jusqu'à 4.6 incluse)

In [86]:
#Entraînement
from sklearn import svm
svrrbf=svm.SVR(C=2.0, epsilon=0.3,kernel='rbf', random_state=16)
svrrbf.fit(X_train,y_train)

#NB : les hyperparamètres C et epsilon sont choisis par validation croisée ci-dessous

SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=16,
  shrinking=True, tol=0.001, verbose=False)

In [87]:
svrrbf.score(X_train, y_train)
#On observe une augmentation du score en prenant 2 au lien de 1 (valeur par défaut) comme valeur de C

0.81390272520296747

In [88]:
#Prédiction
predicted_svrrbf = svrrbf.predict(X_test)
expected_svrrbf = y_test

In [89]:
#Taux de bonnes réponses
nb_goodpred_svrrbf=0
for i in range(0,len(predicted_svrrbf)) :
    if round(predicted_svrrbf[i])==expected_svrrbf[i] :
        nb_goodpred_svrrbf=nb_goodpred_svrrbf+1
print(nb_goodpred_svrrbf/(len(predicted_svrrbf)+1))

0.6076672104404568


In [90]:
#Taux de bonnes réponses sur les vins moyens (qualités 5, 6 et 7)

nb_goodpred_svrrbf_commonwines=0
for i in range(0,len(predicted_svrrbf)) :
    if round(predicted_svrrbf[i])==expected_svrrbf[i] & 4<expected_svrrbf[i]<8 :
        nb_goodpred_svrrbf_commonwines=nb_goodpred_svrrbf+1
print(nb_goodpred_svrrbf/(list(y_test).count(5)+list(y_test).count(6)+list(y_test).count(7)))


0.6581272084805654


In [91]:
#Erreur quadratique totale
e_svrrbf2=((predicted_svrrbf-expected_svrrbf)**2).sum()
print(e_svrrbf2)

703.545033248


In [92]:
#Erreur absolue totale et moyenne
e_svrrbf1=(abs(predicted_svrrbf-expected_svrrbf)).sum()
print(e_svrrbf1)
e_svrrbf1_mean = e_svrrbf1/len(y_test)
print(e_svrrbf1_mean)

698.611861709
0.570295397314


In [93]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 0.5
e_svrrbf_epsilon1=sum(abs(predicted_svrrbf[i]-expected_svrrbf[i]) for i in range(0, len(y_test)) if abs(predicted_svrrbf[i]-expected_svrrbf[i])>0.5)
print(e_svrrbf_epsilon1)

495.810611618


In [94]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 1
e_svrrbf_epsilon2=sum(abs(predicted_svrrbf[i]-expected_svrrbf[i]) for i in range(0, len(y_test)) if abs(predicted_svrrbf[i]-expected_svrrbf[i])>1)
print(e_svrrbf_epsilon2)

294.40939311


In [95]:
#Graphe hexbin : Comparaison des prédictions aux qualités réelles
plt.hexbin(expected_svrrbf,predicted_svrrbf, cmap=plt.cm.YlGn)
#plt.axis([3, 9, 3, 9])
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.plot([4.3,7.7],[4.3,7.7],'--k')
plt.show()

In [96]:
#Introduction de bruit sur les qualités "vraies" pour mieux voir les graphes et arrondi des préditions à un chiffre après la virgule
expected_svrrbf_noise=expected_svrrbf+np.random.uniform(0,0.2,size=len(y_test))

predicted_svrrbf_simplified=np.around(predicted_svrrbf,1)

In [97]:
#Graphe scatter avec du bruit et arrondi à 0.1 près des prédictions (pour mieux voir les prédictions des qualités extrêmes)
ppl.scatter(expected_svrrbf_noise,predicted_svrrbf_simplified)
plt.axis([2.5, 9.5, 2.5, 9.5])
plt.title("Comparaison prédiction vs qualité réelle")
plt.plot([2.5,9.5],[2.5,9.5],'--k')
plt.show()

In [98]:
#Graphe erreur selon taux d'alcool : il semble que les vins avec un taux d'alcool élevé sont mal prédits
ppl.scatter(x=X_test[:,10],y=abs(expected_svrrbf-predicted_svrrbf), color='indianred')
plt.axis('tight')
plt.xlabel('Alcohol')
plt.ylabel('Erreur de prédiction')
plt.title('Alcohol')
plt.show()

In [99]:
#Distribution des taux d'alcool dans la base
pandas.DataFrame.hist(dfwhite, column="alcohol", bins=50, color='indianred', grid=False)
plt.show()

In [100]:
#Graphe erreur selon densité : il semble que les vins avec une faible densité sont plus mal prédits
ppl.scatter(x=X_test[:,7],y=abs(expected_svrrbf_noise-predicted_svrrbf_simplified), color='green')
plt.axis('tight')
plt.xlabel('Density')
plt.ylabel('Erreur en valeur absolue')
plt.title('Density')
plt.show()

In [101]:
#Graphe erreur selon residual sugar : il semble que les vins avec peu de sucre sont moins bien prédits
ppl.scatter(x=X_test[:,3],y=abs(expected_svrrbf_noise-predicted_svrrbf_simplified), color='lightslategray')
plt.axis('tight')
plt.xlabel('Residual sugar')
plt.ylabel('Erreur en valeur absolue')
plt.title('Residual sugar')
plt.show()

In [102]:
#Graphe erreur à décliner selon les autres variables
#Les autres variables donnent des résultats moins intéressants
ppl.scatter(x=X_test[:,9],y=abs(expected_svrrbf_noise-predicted_svrrbf_simplified), color='blue')
plt.axis('tight')
plt.xlabel('Sulphates')
plt.ylabel('Erreur en valeur absolue')
plt.title('Sulphates')
plt.show()

In [103]:
#Prédiction selon le taux d'alcool : les vins à taux d'alcool plus élevé sont prédits avec une qualité supérieure
plt.scatter(expected_svrrbf_noise, predicted_svrrbf_simplified, c=X_test[:,10])
plt.plot([3,9],[3,9],'--k')
cb = plt.colorbar()
cb.set_label('Alcohol')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Qualité prédite')
plt.show()

##### Comparaison régression Ridge polynomiale et régression à vecteurs de suppport à noyau gaussien

In [104]:
#Graphe erreur absolue selon la qualité espérée

#Introduction de plus de bruit sur la qualité réelle pour mieux voir
expected_ridgereg_poly_noise2=expected_ridgereg_poly+np.random.uniform(0,0.5,size=len(y_test))
expected_svrrbf_noise2=expected_svrrbf+np.random.uniform(0,0.5,size=len(y_test))

e_ridgereg_poly_plot=abs(predicted_ridgereg_poly-expected_ridgereg_poly)
e_svrrbf_plot=abs(predicted_svrrbf-expected_svrrbf)

ppl.scatter(expected_svrrbf_noise2, e_svrrbf_plot, color='indianred', label='Régression à vecteurs de support')
ppl.scatter(expected_ridgereg_poly_noise2, e_ridgereg_poly_plot,color='green', label='Régression Ridge polynomiale')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur')
plt.legend()
plt.show()

In [105]:
#Graphe taux d'erreur - évolution
e_sorted_svrrbf=sorted(e_svrrbf_plot)
e_sorted_ridgereg_poly=sorted(e_ridgereg_poly_plot)

ppl.plot(np.arange(0,len(y_test)), e_sorted_svrrbf, color='green')
ppl.plot(np.arange(0,len(y_test)), e_sorted_ridgereg_poly, color='orange')
plt.axis('tight')
plt.xlabel('')
plt.ylabel('Erreur')
plt.show()


##### Grid Search (selon critère de performance par défaut dans scikit learn) pour choisir les hyperparamètres epsilon et C

In [106]:
from sklearn import svm
from sklearn import grid_search

grid = {'C': [0.5,1,2,5,10,50, 100], 'epsilon': [0.1,0.2,0.3,0.5,0.6,0.7,0.8,0.9,1,1.5]}
gd=grid_search.GridSearchCV(estimator=svm.SVR(kernel='rbf'), param_grid=grid, cv=5)

In [ ]:
gd.fit(X_train,y_train)

In [28]:
gd.best_params_

{'C': 1, 'epsilon': 0.3}

##### Recherche de l'hyperparamètre C par validation croisée 5-Fold en essayant de minimiser les erreurs supérieures à 0.5

In [21]:
from sklearn import cross_validation
from sklearn import svm
import numpy as np

kf = cross_validation.KFold(len(y_train), n_folds=5)

hyperparam=[0.1,1.0,2.0,3.0,4.0,5.0,10.0,100,1000]
nb_iter=-1
mat_error=np.zeros((5,len(hyperparam)))

for train_index, test_index in kf:
    nb_iter=nb_iter+1
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_v, X_valid = X_train[train_index], X_train[test_index]
    y_train_v, y_valid = y_train[train_index], y_train[test_index]
    #On entraîne le modèle pour toutes les valeurs d'hyperparamètres sur chaque base d'apprentissage et on prédit les qualités sur chaque base de validation
    for j in range(0,len(hyperparam)):
        svr_bis=svm.SVR(C=hyperparam[j],kernel='rbf', epsilon=0.1)
        svr_bis.fit(X_train_v,y_train_v)
        predicted_svr_bis=svr_bis.predict(X_valid)
        expected_svr_bis=y_valid
        e_svr_bis=abs(predicted_svr_bis-expected_svr_bis)
        #On ne décompte que les erreurs absolues supérieures à 0.5 : (les autres comptent pour 0)
        for k in range(0,len(e_svr_bis)):
            if e_svr_bis[k]<0.5:
                e_svr_bis[k]=0
        mat_error[nb_iter,j]=e_svr_bis.sum()

print(mat_error)
#On obtient une matrice avec en colonnes les différentes valeurs de C et en ligne les différents couples (base d'apprentissage, base de validation)
#Il s'agit donc d'une matrice à 9 colonnes (nombre d'hyperparamètres C testés) et 5 lignes (nombre de couples, car c'est une 5-Fold Cross-Validation)

[[ 423.98099214  332.06285514  327.78732341  323.30129518  328.21594039
   334.02911909  342.14213022  355.63336055  355.64661697]
 [ 429.65640689  351.90781643  343.89362992  348.10773268  351.17542989
   353.38733513  363.0016949   371.65253798  371.67535989]
 [ 397.26808553  302.49605823  304.24697251  301.90533789  306.30772221
   308.32944547  319.92945651  342.6932866   342.70713224]
 [ 414.12409568  328.21096399  326.47314697  329.59199857  328.67359699
   328.34073722  334.21936203  344.04131358  344.04131358]
 [ 435.26505291  323.297916    322.96112763  328.97374987  331.15149946
   336.0522446   348.92226039  365.82274723  365.87065835]]


In [22]:
#Pour chaque valeur de C, on somme les erreurs obtenues sur chacune des 5 bases de validation
mean_error=np.zeros((1,len(hyperparam)))
for i in range (0,len(hyperparam)):
    mean_error[0,i]=mat_error[:,i].sum()
print(mean_error)

[[ 2100.29463315  1637.97560978  1625.36220044  1631.88011419
   1645.52418894  1660.13888151  1708.21490405  1779.84324594
   1779.94108101]]


In [14]:
#On choisit l'hyperparamètre C qui a la somme des erreurs sur chacune des bases de validation la plus petite (ceci est équivalent à choisir celui qui a la moyenne des erreurs sur les 5 bases de test la plus petite)
best_param=0
for i in range (0,len(hyperparam)):
    if mean_error[0,i]==min(mean_error[0,:]):
        best_param=hyperparam[i]
print(best_param)

2.0


- En minimisant les erreurs supérieures à 0.5 :On trouve C=2 pour epsilon=0.1 (avec erreur moyenne sur toutes les bases de test = 1625.3)
- En minimisant les erreurs supérieures à 0.5 :On trouve C=2 pour epsilon=0.3 (avec erreur moyenne sur toutes les bases de test= 1558.3)
- En minimisant les erreurs supérieures à 1 :On trouve C=2 pour epsilon=0.3
- En minimisant les erreurs supérieures à 1 :On trouve C=1 pour epsilon=0.1

### Double modèle selon le taux d'alcool du vin dont la qualité est à prédire (Chapitre 4, partie 4.7)

In [5]:
#Division de la base initiale en une base qui contient les vins à plus de 12 degrés d'alcool et une base qui contient les vins à moins de 12 degrés d'alcool)
df_high=dfwhite[dfwhite.alcohol>12]
df_low=dfwhite[dfwhite.alcohol<=12]

In [164]:
df_high.shape, df_low.shape

((711, 12), (4187, 12))

In [6]:
#Entraînement sur chacune des 2 nouvelles bases d'apprentissage
from sklearn.cross_validation import train_test_split

X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(df_high[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']],df_high.quality, random_state=3)

X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(df_low[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']],df_low.quality, random_state=3)


In [11]:
#Création d'une nouvelle "grande" base d'apprentissage et "grande" base de test contenant des vins à taux d'alcool quelconque pour comparer ensuite les résultats sur la même base de test
new_X_train =np.concatenate((X_train_low,X_train_high), axis=0)
new_X_test=np.concatenate((X_test_low,X_test_high), axis=0)
new_y_train=np.concatenate((y_train_low,y_train_high), axis=0)
new_y_test=np.concatenate((y_test_low,y_test_high), axis=0)


##### Sur la nouvelle "grande" base de test et apprentissage (uniquement pour comparer les résultats, on doit normalement retrouver des résultats proches de ci-dessus) : entraînement d'un modèle SVR à noyau gaussien et prédiction sur la nouvelle grande base de test

In [12]:
#Entraînement
from sklearn import svm
svrrbf=svm.SVR(C=2.0, epsilon=0.3,kernel='rbf', random_state=16)
svrrbf.fit(new_X_train,new_y_train)

SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=16,
  shrinking=True, tol=0.001, verbose=False)

In [13]:
svrrbf.score(new_X_train, new_y_train)

0.82316858879938648

In [14]:
#Prédiction
predicted_svrrbf_new = svrrbf.predict(new_X_test)
expected_svrrbf_new = new_y_test

In [15]:
#Taux de bonnes réponses
nb_goodpred_svrrbf_new=0
for i in range(0,len(predicted_svrrbf_new)) :
    if round(predicted_svrrbf_new[i])==expected_svrrbf_new[i] :
        nb_goodpred_svrrbf_new=nb_goodpred_svrrbf_new+1
print(nb_goodpred_svrrbf_new/(len(predicted_svrrbf_new)+1))

0.5864600326264274


In [16]:
#Erreur quadratique totale
e_svrrbf2_new=((predicted_svrrbf_new-expected_svrrbf_new)**2).sum()
print(e_svrrbf2_new)

709.581447477


In [17]:
#Erreur absolue totale
e_svrrbf1_new=(abs(predicted_svrrbf_new-expected_svrrbf_new)).sum()
print(e_svrrbf1_new)

711.198049746


In [18]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 0.5
e_svrrbf_epsilon1_new=sum(abs(predicted_svrrbf_new[i]-expected_svrrbf_new[i]) for i in range(0, len(new_y_test)) if abs(predicted_svrrbf_new[i]-expected_svrrbf_new[i])>0.5)
print(e_svrrbf_epsilon1_new)

514.557036435


In [19]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 1
e_svrrbf_epsilon2_new=sum(abs(predicted_svrrbf_new[i]-expected_svrrbf_new[i]) for i in range(0, len(new_y_test)) if abs(predicted_svrrbf_new[i]-expected_svrrbf_new[i])>1)
print(e_svrrbf_epsilon2_new)

295.079647656


##### Sur la base low

In [21]:
#Entraînement
from sklearn import svm
svrrbf_low=svm.SVR(C=2.0, epsilon=0.2,kernel='rbf', random_state=16)
#Les hyperparamètres sont choisis selon la validation croisée effectuée plus bas
svrrbf_low.fit(X_train_low,y_train_low)

SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=16,
  shrinking=True, tol=0.001, verbose=False)

In [22]:
svrrbf_low.score(X_train_low,y_train_low)

0.85975390843562893

In [23]:
#Prédiction
predicted_low=svrrbf_low.predict(X_test_low)
expected_low=y_test_low

In [24]:
#Nombre de bonnes réponses sur la base de test low
nb_goodpred_low=0
for i in range(0,len(predicted_low)) :
    if round(predicted_low[i])==expected_low[i] :
        nb_goodpred_low=nb_goodpred_low+1
print(nb_goodpred_low)

624


##### Sur la base high

In [49]:
#Entraînement
from sklearn import svm

svrrbf_high=svm.SVR(C=0.5, epsilon=0.5,kernel='rbf', random_state=16)
#Les hyperparamètres sont choisis selon la validation croisée effectuée plus bas

svrrbf_high.fit(X_train_high,y_train_high)

SVR(C=0.5, cache_size=200, coef0=0.0, degree=3, epsilon=0.5, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=16,
  shrinking=True, tol=0.001, verbose=False)

In [50]:
svrrbf_high.score(X_train_high,y_train_high)

0.36213979481467407

In [51]:
#Prédiction
predicted_high=svrrbf_high.predict(X_test_high)
expected_high=y_test_high

In [52]:
#Nombre de bonnes réponses sur la base high
nb_goodpred_high=0
for i in range(0,len(predicted_high)) :
    if round(predicted_high[i])==expected_high[i] :
        nb_goodpred_high=nb_goodpred_high+1
print(nb_goodpred_high)

81


In [53]:
#Graphe hexbin : comparaison des prédictions aux qualités réelles
#On n'observe déjà qu'il n'y a pas vraiment d'amélioration
plt.hexbin(expected_high,predicted_high, cmap=plt.cm.YlGn)
plt.title("Comparaison prédiction vs qualité réelle")
cb = plt.colorbar()
cb.set_label('Nombre de prédictions')
plt.show()

##### Erreurs totales (sur base de test low + base de test high = "grande" base de test)

In [54]:
#Total de bonnes prédictions sur la "grande" base de test avec le double modèle
nb_goodpred_total=(nb_goodpred_high+nb_goodpred_low)/(len(y_test_high)+len(y_test_low))
print(nb_goodpred_total)

0.5755102040816327


In [55]:
#Erreur absolue totale sur la "grande" base de test avec le double modèle
e_total1=(abs(predicted_high-expected_high)).sum()+(abs(predicted_low-expected_low)).sum()
print(e_total1)

682.727607796


In [56]:
#Erreur quadratique totale sur la "grande" base de test avec le double modèle
e_total2=((predicted_high-expected_high)**2).sum()+((predicted_low-expected_low)**2).sum()
print(e_total2)

671.526759722


In [57]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 0.5
e_high_epsilon1=sum(abs(predicted_high[i]-expected_high[i]) for i in range(0, len(y_test_high)) if abs(predicted_high[i]-expected_high[i])>0.5)
e_low_epsilon1=sum(abs(predicted_low[i]-expected_low[i]) for i in range(0, len(y_test_low)) if abs(predicted_low[i]-expected_low[i])>0.5)
e_total_epsilon1=e_high_epsilon1+e_low_epsilon1
print(e_total_epsilon1)

504.531002513


In [58]:
#Erreur sans prendre en compte les erreurs en valeur absolue inférieures à epsilon = 1
e_high_epsilon2=sum(abs(predicted_high[i]-expected_high[i]) for i in range(0, len(y_test_high)) if abs(predicted_high[i]-expected_high[i])>1)
e_low_epsilon2=sum(abs(predicted_low[i]-expected_low[i]) for i in range(0, len(y_test_low)) if abs(predicted_low[i]-expected_low[i])>1)
e_total_epsilon2=e_high_epsilon2+e_low_epsilon2
print(e_total_epsilon2)

284.646865408


##### Gridsearch pour trouver les hyperparamètres sur les bases low et high

In [168]:
from sklearn import svm
from sklearn import grid_search

grid = {'C': [0.5,1,2,5,10,50, 100], 'epsilon': [0.1,0.2,0.3,0.5,0.6,0.7,0.8,0.9,1,1.5]}
gd=grid_search.GridSearchCV(estimator=svm.SVR(kernel='rbf'), param_grid=grid)

In [171]:
gd.fit(X_train_high,y_train_high)
gd.best_params_

{'C': 0.5, 'epsilon': 0.5}

In [172]:
gd.fit(X_train_low,y_train_low)
gd.best_params_

{'C': 2, 'epsilon': 0.2}

##### Comparaison entre le modèle SVR simple et le double modèle selon le taux d'alcool

In [59]:
#Graphe erreur absolue selon la qualité espérée

expected_svrrbf_new_noise=expected_svrrbf_new+np.random.uniform(0,0.5,size=len(new_y_test))
expected_low_noise=expected_low+np.random.uniform(0,0.5,size=len(y_test_low))
expected_high_noise=expected_high+np.random.uniform(0,0.5,size=len(y_test_high))


import prettyplotlib as ppl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
ppl.scatter(expected_svrrbf_new_noise, abs(expected_svrrbf_new-predicted_svrrbf_new), color='indianred', label='SVR sur base complète')
ppl.scatter(expected_low_noise, abs(expected_low-predicted_low),color='green', label='SVR sur base de vins à faible taux dalcool')
ppl.scatter(expected_high_noise, abs(expected_high-predicted_high),color='green', label='SVR sur base de vins à fort taux dalcool')
plt.axis('tight')
plt.xlabel('Vraie qualité')
plt.ylabel('Erreur')
plt.legend()
plt.show()

# ANNEXE : Problème de classification

J'ai commencé à réfléchir ici à un éventuel moyen de détecter les mauvais vins par classification binaire. J'ai considéré les vins de qualités 3 et 4 comme ds "mauvais" vin (sachant qu'il n'y a pas de qualités strictement inférieures à 3).

##### Transformation de la variable quality en une variable binaire indiquant si le vin est mauvais ou pas

In [2]:
dfwhite_2=dfwhite.copy()
dfwhite_2['poor wine']=dfwhite_2['quality'].apply(lambda x: 1 if x<5 else -1)
dfwhite_2=dfwhite_2.drop('quality', axis=1)

In [3]:
dfwhite_2.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,poor wine
4893,6.2,0.21,0.29,1.6,0.039,24,92,0.99114,3.27,0.50,11.2,-1
4894,6.6,0.32,0.36,8.0,0.047,57,168,0.99490,3.15,0.46,9.6,-1
4895,6.5,0.24,0.19,1.2,0.041,30,111,0.99254,2.99,0.46,9.4,-1
4896,5.5,0.29,0.30,1.1,0.022,20,110,0.98869,3.34,0.38,12.8,-1
4897,6.0,0.21,0.38,0.8,0.020,22,98,0.98941,3.26,0.32,11.8,-1


In [4]:
dfwhite_2['poor wine'].value_counts()
#On remarque qu'on a très peu de mauvais vins, les classes sont disproportionnées.

-1    4715
 1     183
dtype: int64

##### Séparation entre une base de test (25%) et une base d'entraînement (75%) pour prédiction de la variable 'poor wine'

In [5]:
from sklearn.cross_validation import train_test_split

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(dfwhite_2[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']],dfwhite_2['poor wine'], random_state=3)

### Analyse discriminante linéaire

Dans l'anayse discriminante, si une classe est sous-représentée ce n'est pas un problème car on apprend à prédire une classe contre l'autre.

In [6]:
from sklearn import lda

linearda=lda.LDA()

linearda.fit(X_train_2,y_train_2)

LDA(n_components=None, priors=None)

In [7]:
linearda.means_

array([[  6.84691131e+00,   2.74641542e-01,   3.35803344e-01,
          6.51734202e+00,   4.59832814e-02,   3.55508643e+01,
          1.38025928e+02,   9.94051547e-01,   3.18597620e+00,
          4.90427883e-01,   1.05139265e+01],
       [  7.26111111e+00,   3.81250000e-01,   3.09861111e-01,
          4.45000000e+00,   5.01180556e-02,   2.57187500e+01,
          1.27180556e+02,   9.94181806e-01,   3.17611111e+00,
          4.83333333e-01,   1.02322917e+01]])

In [8]:
predicted_linearda=linearda.predict(X_test_2)
expected_linearda=y_test_2

In [9]:
#Taux de bonnes réponses
#Ca n'indique en fait ici pas grand chose puisque comme la classe des mauvais vins est sous-représentée, le classifier obtient un taux de bonnes réponses très élevée en ne prédisant que "bon" et jamais "mauvais"
nb_goodpred_linearda=0
for i in range(0,len(predicted_linearda)) :
    if predicted_linearda[i]==expected_linearda[i]:
        nb_goodpred_linearda=nb_goodpred_linearda+1
print(nb_goodpred_linearda/(len(predicted_linearda)+1))

0.9608482871125612


In [10]:
#Nombres de mauvais vins prédits "mauvais"
nb_goodpred_linearda_poor=0
for i in range(0,len(predicted_linearda)) :
    if predicted_linearda[i]==expected_linearda[i] and expected_linearda[i]==1 :
        nb_goodpred_linearda_poor=nb_goodpred_linearda_poor+1
print(nb_goodpred_linearda_poor)
#C'est bien ce que donne la matrice de confusion ci-dessous

6


In [11]:
#Matrice de confusion
from sklearn.metrics import confusion_matrix

cm_linearda = confusion_matrix(expected_linearda,predicted_linearda)
print(cm_linearda)
import matplotlib.pyplot as plt
plt.matshow(cm_linearda)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

[[1172   14]
 [  33    6]]


In [12]:
#Nombre de "vrais" mauvais vins dans la base de test
(y_test_2==1).sum()

39

In [13]:
#On a donc détecté ce pourcentage de mauvais vins:
cm_linearda[1,1]/(cm_linearda[1,0]+cm_linearda[1,1])

0.15384615384615385

Il détecte mal les mauvais vins alors que c'est ce qui nous intéresse. Heureusement, ce classifier prédit peu de bons vins comme étant mauvais (14 selon la matrice de confusion). Ainsi, si l'on imagine un programme qui évite de goûter tous les vins, goûter seulement 14 + 6 vins est vraiment peu coûteux.

##### Courbe ROC

NB : Dans un test de classification binaire, un résultat est dit vrai positif lorsqu'un item est correctement détecté par le test. On l'oppose aux notions de faux positif (item déclaré positif alors qu'il ne l'était pas), de faux négatif (item déclaré négatif alors qu'il était en réalité positif) et de vrai négatif (item correctement déclaré comme négatif).

Dans le cas présent, idéalement on aimerait obtenir 0 faux négatif (aucun vin mauvais prédit "bon" c'est-à-dire prédit -1) et à ce moment là le nombre de vrai négatif est moins important car dans tous les cas on devra goûter moins de vins que si on les goûtait tous pour trouver les mauvais. 

In [24]:
from sklearn import metrics
y_score = linearda.fit(X_train_2, y_train_2).decision_function(X_test_2)
fpr, tpr, thresholds = metrics.roc_curve(y_test_2,y_score)
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
#fraction des Négatifs classés positifs
plt.ylabel('True Positive Rate')
#fraction des Positifs classés positifs
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

Plus l'aire sous la courbe est grande, plus elle se rapproche du classifier idéal (au point (0,1))

Remarque : On voit sur cette courbe ROC qu'en mettant fixant arbitrairement un certain seuil de prédiction, on pourrait détecter environ 90% des mauvais vins (true positive rate = 0.9 ) en goûtant beaucoup moins de vins que la totalité puisque le false positive rate est environ à 0.65 pour ce true positive rate. On serait donc obligés de goûter environ 65% des vins qui en réalité sont bons au lieu de 100%, ce qui entraînerait une forte diminution des coûts.

### Quadratic discriminant analysis

Idem pas de problème de sous-représentation ici.

In [25]:
from sklearn import qda

quadrada=qda.QDA()

quadrada.fit(X_train_2,y_train_2)

QDA(priors=None, reg_param=0.0)

In [26]:
predicted_quadrada=quadrada.predict(X_test_2)
expected_quadrada=y_test_2

In [27]:
from sklearn.metrics import confusion_matrix

cm_quadrada = confusion_matrix(expected_quadrada,predicted_quadrada)
print(cm_quadrada)
import matplotlib.pyplot as plt
plt.matshow(cm_quadrada)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

[[1156   30]
 [  29   10]]


In [28]:
#On a donc détecté ce pourcentage de mauvais vins:
cm_quadrada[1,1]/(cm_quadrada[1,0]+cm_quadrada[1,1])

0.25641025641025639

Il y a une légère amélioration par rapport à l'analyse discriminante linéaire : on détecte 1/4 des mauvais vins. Et on prédit 40 mauvais vins, c'est assez peu donc c'est plutôt un bon résultat. Mais l'air en-dessous de la courbe ROC est légèrement moins grande.

In [36]:
#Courbe ROC
from sklearn import metrics
y_score = quadrada.fit(X_train_2, y_train_2).decision_function(X_test_2)
fpr, tpr, thresholds = metrics.roc_curve(y_test_2,y_score)
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
#fraction des Négatifs classés positifs
plt.ylabel('True Positive Rate')
#fraction des Positifs classés positifs
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

### Machine à Vecteurs de support - Classification

Avec cette méthode, il faudrait plus réfléchir au problème de sous-représentation.

##### SVM Linéaire

In [30]:
from sklearn import svm
#class_weight : {dict, ‘auto’}, optional
#Set the parameter C of class i to class_weight[i]*C for SVC. If not given, all classes are supposed to have weight one. The ‘auto’ mode uses the values of y to automatically adjust weights inversely proportional to class frequencies.

#On donne plus de poids à la classe sous-représentée car c'est celle qui nous intéresse
svm_lin=svm.SVC(C=0.5,kernel='linear', class_weight='auto')
svm_lin.fit(X_train_2,y_train_2)

SVC(C=0.5, cache_size=200, class_weight='auto', coef0=0.0, degree=3,
  gamma=0.0, kernel='linear', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [31]:
predicted_svmlin=svm_lin.predict(X_test_2)
expected_svmlin=y_test_2

In [32]:
from sklearn.metrics import confusion_matrix

cm_svmlin = confusion_matrix(expected_svmlin,predicted_svmlin)
print(cm_svmlin)
import matplotlib.pyplot as plt
plt.matshow(cm_svmlin)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

[[952 234]
 [ 18  21]]


In [33]:
#On a donc détecté ce pourcentage de mauvais vins:
cm_svmlin[1,1]/(cm_svmlin[1,0]+cm_svmlin[1,1])

0.53846153846153844

On détecte 50% de vrais mauvais vins sauf que on détecte 224 autres mauvais vins qui en réalité ne le sont pas, ce qui très coûteux comparé au nombre de vins total, surtout si parmi ces 224 il n'y a que la moitié des mauvais vins.

On obtient une aire sous la courbe ROC légèrement moins grande que l'analyse discriminante linéaire.

In [35]:
#Courbe ROC
from sklearn import metrics
y_score = svm_lin.fit(X_train_2, y_train_2).decision_function(X_test_2)
fpr, tpr, thresholds = metrics.roc_curve(y_test_2,y_score)
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
#fraction des Négatifs classés positifs
plt.ylabel('True Positive Rate')
#fraction des Positifs classés positifs
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

K-fold cross-validation sur l'hyperparamètre C

In [7]:
from sklearn import cross_validation
from sklearn import svm
from sklearn.metrics import confusion_matrix

kf = cross_validation.KFold(len(y_train_2), n_folds=5)

hyperparam=[0.1,0.5,1.0,3,10.0,20.0]
nb_iter=-1
mat_error=np.zeros((5,len(hyperparam)))

for train_index, test_index in kf:
    nb_iter=nb_iter+1
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_v, X_valid = X_train_2[train_index], X_train_2[test_index]
    y_train_v, y_valid = y_train_2[train_index], y_train_2[test_index]
    #On entraîne le modèle pour toutes les valeurs d'hyperparamètres sur la base d'apprentissage et la base de validation obtenues
    for j in range(0,len(hyperparam)):
        svm_lin1=svm.SVC(C=hyperparam[j],kernel='linear', class_weight='auto')
        svm_lin1.fit(X_train_v,y_train_v)
        predicted_svmlin1=svm_lin1.predict(X_valid)
        expected_svmlin1=y_valid
        cm_svmlin1 = confusion_matrix(expected_svmlin1,predicted_svmlin1)
        mat_error[nb_iter,j]=cm_svmlin1[0,1]+cm_svmlin1[1,0]

print(mat_error)

#NB : algo brute force : on essaye toutes les possibilités une par une


[[ 177.  152.  148.  138.  139.  146.]
 [ 158.  122.  131.  149.  158.  159.]
 [ 168.  134.  132.  141.  148.  154.]
 [ 212.  200.  199.  202.  198.  198.]
 [ 199.  169.  176.  178.  181.  191.]]


In [13]:
mean_error=np.zeros((1,len(hyperparam)))
for i in range (0,len(hyperparam)):
    mean_error[0,i]=mat_error[:,i].sum()
print(mean_error)


[[ 914.  777.  786.  808.  824.  848.]]


In [15]:
best_param=0
for i in range (0,len(hyperparam)):
    if mean_error[0,i]==min(mean_error[0,:]):
        best_param=hyperparam[i]
print(best_param)

0.5
